# Analyse des attaquants — KPIs, méthodes, visualisations
Notebook pédagogique et analytique basé sur FBref (Big‑5, 2024‑25) et colonnes personnalisées.

## 0. Chargement des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 150)
df = pd.read_csv(r"/mnt/data/assembled_data_FW_normalized.csv")
df.head(3)

## 1. Audit du CSV et filtrage de stabilité
On applique un seuil de 600 minutes pour limiter le bruit.

In [ ]:
df = df.copy()
df['minutes'] = df['90s'] * 90
base = df[df['minutes'] >= 600].copy()
base.shape

In [ ]:
def bar_top(df_in, col, n=15, title=None):
    t = df_in[['Player', col]].dropna().sort_values(col, ascending=False).head(n)
    fig, ax = plt.subplots(figsize=(9,6))
    ax.bar(t['Player'], t[col])
    ax.set_title(title or col)
    ax.set_xticklabels(t['Player'], rotation=75, ha='right')
    ax.set_ylabel(col)
    plt.tight_layout()
    plt.show()

def hist_plot(series, title=None, bins=20):
    s = series.dropna()
    fig, ax = plt.subplots(figsize=(8,5))
    ax.hist(s, bins=bins)
    ax.set_title(title or series.name)
    ax.set_xlabel(series.name)
    ax.set_ylabel("Fréquence")
    plt.tight_layout()
    plt.show()

### KPI : npG/90
**Pourquoi**: Mesure la finition hors pénalty. Compare des finisseurs sans inflation liée aux pénos.

**Formule**: `npG/90 = (Gls − PK) / 90s`

In [ ]:
base['npG'] = base['Gls'] - base.get('PK', 0)
base['npG_per90'] = base['npG'] / base['90s']
bar_top(base, 'npG_per90', title='npG/90 — Top 15')
base[['Player','Squad','Comp','npG_per90']].sort_values('npG_per90', ascending=False).head(10)

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Finishing_Delta_np
**Pourquoi**: Sur/sous-performance de finition vs attentes.

**Formule**: `Δ = (Gls − PK) − npxG`

In [ ]:
base['npG'] = base['Gls'] - base.get('PK', 0)
base['Finishing_Delta_np'] = base['npG'] - base['npxG']
bar_top(base, 'Finishing_Delta_np', title='Finishing Δ (npG − npxG) — Top 15')
base[['Player','Squad','Comp','Finishing_Delta_np']].sort_values('Finishing_Delta_np', ascending=False).head(10)

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Finishing_Ratio_np
**Pourquoi**: Étalonne l’écart de finition par la difficulté des tirs.

**Formule**: `Ratio = (Gls − PK) / npxG`

In [ ]:
base['npG'] = base['Gls'] - base.get('PK', 0)
base['Finishing_Ratio_np'] = base['npG'] / base['npxG']
series = base['Finishing_Ratio_np'].replace([np.inf, -np.inf], np.nan).dropna()
hist_plot(series, title='Finishing Ratio (npG / npxG) — Distribution')
base[['Player','Squad','Comp','Finishing_Ratio_np']].sort_values('Finishing_Ratio_np', ascending=False).head(10)

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Shot_Accuracy
**Pourquoi**: Qualité de cadrage.

**Formule**: `SoT / Sh`

In [ ]:
base['Shot_Accuracy'] = base['SoT'] / base['Sh_shooting']
hist_plot(base['Shot_Accuracy'], title='Shot Accuracy — Distribution')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Shot_Conversion
**Pourquoi**: Efficacité au tir.

**Formule**: `Gls / Sh`

In [ ]:
base['Shot_Conversion'] = base['Gls'] / base['Sh_shooting']
hist_plot(base['Shot_Conversion'], title='Shot Conversion — Distribution')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : npxG_per_Shot
**Pourquoi**: Qualité moyenne des tirs.

**Formule**: `npxG / Sh`

In [ ]:
base['npxG_per_Shot'] = base['npxG'] / base['Sh_shooting']
hist_plot(base['npxG_per_Shot'], title='npxG par tir — Distribution')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Box_Touches_per90
**Pourquoi**: Présence dans la surface.

**Formule**: `Att Pen_per_90`

In [ ]:
base['Box_Touches_per90'] = base.get('Att Pen_per_90', 0)
bar_top(base, 'Box_Touches_per90', title='Touches surface/90 — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Box_Touch_Share
**Pourquoi**: Orientation vers la zone de but.

**Formule**: `Att Pen / Touches`

In [ ]:
base['Box_Touch_Share'] = base.get('Att Pen', 0) / base.get('Touches', 1)
hist_plot(base['Box_Touch_Share'], title='Part des touches en surface — Distribution')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Prog_Reception_per90
**Pourquoi**: Capacité à recevoir haut.

**Formule**: `PrgR_per_90`

In [ ]:
base['Prog_Reception_per90'] = base.get('PrgR_per_90', 0)
bar_top(base, 'Prog_Reception_per90', title='Réceptions progressives/90 — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Prog_Carries_per90
**Pourquoi**: Menace balle au pied.

**Formule**: `PrgC_per_90`

In [ ]:
base['Prog_Carries_per90'] = base.get('PrgC_per_90', 0)
bar_top(base, 'Prog_Carries_per90', title='Conduites progressives/90 — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Carries_into_Box_per90
**Pourquoi**: Pénétration surface.

**Formule**: `CPA_per_90`

In [ ]:
base['Carries_into_Box_per90'] = base.get('CPA_per_90', 0)
bar_top(base, 'Carries_into_Box_per90', title='Entrées balle au pied en surface/90 — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Crosses_into_Box_per90
**Pourquoi**: Centres dangereux.

**Formule**: `CrsPA_per_90`

In [ ]:
base['Crosses_into_Box_per90'] = base.get('CrsPA_per_90', 0)
bar_top(base, 'Crosses_into_Box_per90', title='Centres vers la surface/90 — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : xGI_per90
**Pourquoi**: Menace offensive totale attendue.

**Formule**: `xG_per_90 + xAG_per_90`

In [ ]:
base['xGI_per90'] = base.get('xG_per_90', 0) + base.get('xAG_per_90', 0)
bar_top(base, 'xGI_per90', title='xGI/90 — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : A_minus_xAG_per90
**Pourquoi**: Sur/sous-conversion des receveurs.

**Formule**: `A-xAG_per_90`

In [ ]:
base['A_minus_xAG_per90'] = base.get('A-xAG_per_90', 0)
hist_plot(base['A_minus_xAG_per90'], title='A − xAG (per 90) — Distribution')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : Shots_per_BoxTouch
**Pourquoi**: Agressivité dans la surface.

**Formule**: `Sh / Att Pen`

In [ ]:
base['Shots_per_BoxTouch'] = base['Sh_shooting'] / base.get('Att Pen', 1)
hist_plot(base['Shots_per_BoxTouch'], title='Tirs par touche en surface — Distribution')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

### KPI : GA_minus_xGA_np_per90
**Pourquoi**: Durabilité probable de la production hors pénos.

**Formule**: `RealGA_np/90 − (npxG/90 + xAG/90)`

In [ ]:
if 'G+A-PK_per_90' in base.columns:
    base['RealGA_np_per90'] = base['G+A-PK_per_90']
else:
    base['RealGA_np_per90'] = (base['Gls'] + base['Ast'] - base.get('PK',0)) / base['90s']
base['xGA_np_per90'] = base.get('npxG_per_90', 0) + base.get('xAG_per_90', 0)
base['GA_minus_xGA_np_per90'] = base['RealGA_np_per90'] - base['xGA_np_per90']
bar_top(base, 'GA_minus_xGA_np_per90', title='Durabilité réelle − attendue (np) — Top 15')

**Interprétation**: valeurs élevées ⇒ meilleure performance pour ce KPI. Toujours croiser avec rôle et minutes.

## 3. Colonnes personnalisées
Résumé des principales colonnes créées dans ce notebook.

In [ ]:
custom_desc = {
    "npG": "Buts hors pénalty = Gls − PK",
    "npG_per90": "npG ramené à 90 minutes",
    "Shot_Accuracy": "SoT / Sh_shooting",
    "Shot_Conversion": "Gls / Sh_shooting",
    "npxG_per_Shot": "npxG / Sh_shooting",
    "Finishing_Delta_np": "(Gls − PK) − npxG",
    "Finishing_Ratio_np": "(Gls − PK) / npxG",
    "Box_Touches_per90": "Att Pen_per_90",
    "Box_Touch_Share": "Att Pen / Touches",
    "Prog_Reception_per90": "PrgR_per_90",
    "Prog_Carries_per90": "PrgC_per_90",
    "Prog_Passes_per90": "PrgP_per_90",
    "Carries_into_Box_per90": "CPA_per_90",
    "Crosses_into_Box_per90": "CrsPA_per_90",
    "xGI_per90": "xG_per_90 + xAG_per_90",
    "A_minus_xAG_per90": "A-xAG_per_90",
    "Shots_per_BoxTouch": "Sh_shooting / Att Pen",
    "RealGA_np_per90": "(Gls + Ast − PK) / 90s si G+A-PK absente",
    "xGA_np_per90": "npxG_per_90 + xAG_per_90",
    "GA_minus_xGA_np_per90": "RealGA_np_per90 − xGA_np_per90"
}
import pandas as pd
pd.DataFrame([{"Colonne":k, "Définition":v} for k,v in custom_desc.items()])

## 4. Conclusion
- Combiner quantité (xGI/90) et qualité (npxG/Shot, Finishing Δ).
- Filtrer par minutes pour robustesse.
- Interpréter par rôle: 9, ailier, deuxième attaquant.